In [1]:
!pip3 install tensorflow numpy requests tqdm


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from string import punctuation

In [3]:
import requests
content = requests.get("http://www.gutenberg.org/cache/epub/11/pg11.txt").text
open("wonderland.txt", "w", encoding="utf-8").write(content)

167704

In [16]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30
# dataset file path
FILE_PATH = "wonderland.txt"
BASENAME = os.path.basename(FILE_PATH)
# read the data
text = open(FILE_PATH, encoding="utf-8").read()
# remove caps, comment this code if you want uppercase characters as well
text = text.lower()


In [15]:
import string

punctuation = string.punctuation  # Use built-in string.punctuation for all standard punctuation

#replace all occurrences of the letter "a" with the letter "b" in a string, but also removes all punctuation characters.
trans = str.maketrans("a", "b", punctuation)

#
txt = "This is a text with punctuation!"
translated_text = txt.translate(trans)
print(translated_text)  


This is b text with punctubtion


In [17]:
text = text.translate(str.maketrans("", "", punctuation))
text

'\ufeffthe project gutenberg ebook of alices adventures in wonderland\n\n    \n\nthis ebook is for the use of anyone anywhere in the united states and\n\nmost other parts of the world at no cost and with almost no restrictions\n\nwhatsoever you may copy it give it away or reuse it under the terms\n\nof the project gutenberg license included with this ebook or online\n\nat wwwgutenbergorg if you are not located in the united states\n\nyou will have to check the laws of the country where you are located\n\nbefore using this ebook\n\n\n\ntitle alices adventures in wonderland\n\n\n\n\n\nauthor lewis carroll\n\n\n\nrelease date june 27 2008 ebook 11\n\n                most recently updated february 4 2024\n\n\n\nlanguage english\n\n\n\ncredits arthur dibianca and david widger\n\n\n\n\n\n start of the project gutenberg ebook alices adventures in wonderland \n\nillustration\n\n\n\n\n\n\n\n\n\nalice’s adventures in wonderland\n\n\n\nby lewis carroll\n\n\n\nthe millennium fulcrum edition 30\n\n

In [20]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 0123456789abcdefghijklmnopqrstuvwxyzù—‘’“”•™﻿
Number of characters: 161979
Number of unique characters: 47


In [22]:
# dictionary that converts characters to integers
char2int = {c: i for i, c in enumerate(vocab)}
char2int

{'\n': 0,
 ' ': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'k': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'q': 28,
 'r': 29,
 's': 30,
 't': 31,
 'u': 32,
 'v': 33,
 'w': 34,
 'x': 35,
 'y': 36,
 'z': 37,
 'ù': 38,
 '—': 39,
 '‘': 40,
 '’': 41,
 '“': 42,
 '”': 43,
 '•': 44,
 '™': 45,
 '\ufeff': 46}

In [23]:
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(vocab)}
int2char

{0: '\n',
 1: ' ',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9',
 12: 'a',
 13: 'b',
 14: 'c',
 15: 'd',
 16: 'e',
 17: 'f',
 18: 'g',
 19: 'h',
 20: 'i',
 21: 'j',
 22: 'k',
 23: 'l',
 24: 'm',
 25: 'n',
 26: 'o',
 27: 'p',
 28: 'q',
 29: 'r',
 30: 's',
 31: 't',
 32: 'u',
 33: 'v',
 34: 'w',
 35: 'x',
 36: 'y',
 37: 'z',
 38: 'ù',
 39: '—',
 40: '‘',
 41: '’',
 42: '“',
 43: '”',
 44: '•',
 45: '™',
 46: '\ufeff'}

In [7]:
# save these dictionaries for later generation
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [8]:
# convert all text into integers
encoded_text = np.array([char2int[c] for c in text])

In [9]:
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [10]:
# print first 5 characters
for char in char_dataset.take(8):
    print(char.numpy(), int2char[char.numpy()])

38 ﻿
27 p
29 r
26 o
21 j
16 e
14 c
31 t


In [11]:
# build sequences by batching
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)

# print sequences
for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

project gutenbergs alices adventures in wonderland by lewis carroll

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever  you may copy it give it away or

reuse it under the terms of the project gutenberg license included
with this ebook or online at wwwgutenbergorg


title alices adventures in wonderland

author lewis carroll

posting date june 25 2008


In [12]:
def split_sample(sample):
    ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
    for i in range(1, (len(sample)-1) // 2):
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

# prepare inputs and targets
dataset = sequences.flat_map(split_sample)

In [13]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)


dataset = dataset.map(one_hot_samples)

In [14]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: ﻿project gutenbergs alices adventures in wonderland by lewis carroll

this ebook is for the use of a
Target: n
Input shape: (100, 39)
Target shape: (39,)

Input: project gutenbergs alices adventures in wonderland by lewis carroll

this ebook is for the use of an
Target: y
Input shape: (100, 39)
Target shape: (39,)



In [15]:
# repeat, shuffle and batch the dataset
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [16]:
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [17]:
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
# train the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=1)
# save the model
model.save(f"results/{BASENAME}-{sequence_length}.h5")


1209/1209 [==============================] - 48s 40ms/step - loss: 2.2953


In [18]:
import numpy as np
import pickle
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
import os

sequence_length = 100
# dataset file path
FILE_PATH = "wonderland.txt"
# FILE_PATH = "data/python_code.py"
BASENAME = os.path.basename(FILE_PATH)
seed = "chapter xiii"

In [19]:
# load vocab dictionaries
char2int = pickle.load(open(f"/content/{BASENAME}-char2int.pickle", "rb"))
int2char = pickle.load(open(f"/content/{BASENAME}-int2char.pickle", "rb"))
vocab_size = len(char2int)


In [20]:
# building the model
model = Sequential([
    LSTM(256, input_shape=(sequence_length, vocab_size), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(vocab_size, activation="softmax"),
])

In [21]:
# load the optimal weights
model.load_weights(f"/content/results/{BASENAME}-{sequence_length}.h5")

In [22]:
s = seed
n_chars = 400
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
        X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 400/400 [00:14<00:00, 28.40it/s]

Seed: chapter xiii
Generated text:
n and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the work and the 
